In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
#crate paths for csv files
CURR_DIR = os.getcwd()
print(CURR_DIR)
zlecenia_path = os.path.join(CURR_DIR, 'Zlecenia.csv')
godziny_path = os.path.join(CURR_DIR, 'Godziny.csv')

C:\Users\ksobo\Desktop\DS_final_project


In [3]:
#Zaciagniecie baz danych
obciazenie_df = pd.read_csv(zlecenia_path, sep=';')
godziny_df = pd.read_csv(godziny_path, sep=';')

In [4]:
#Ekstrakcja danych i organizacja danych - pozostawienie tylko kolumn wymaganych, pozostawienie tylko projektów "w toku"
obciazenie_df1 = obciazenie_df.drop(columns=['work_preparation_Plan','work_preparation_done'], axis=1)
obciazenie_df2 = obciazenie_df1.loc[obciazenie_df1['Active'].isin(['Active'])]

In [5]:
#Left join po No z godzinami
obciazenie_df2 = pd.merge(obciazenie_df2, godziny_df, how='left', on='No')

#TEST!!!!! wypelnienie NaN zerami
#obciazenie_df2['Wszystkie.technol'] = obciazenie_df2['Wszystkie.technol'].fillna(0)
#obciazenie_df2['Wszystkie.wykonane'] = obciazenie_df2['Wszystkie.wykonane'].fillna(0)

In [6]:
#Zdefiniowanie tygodnia obecnego
#przyjmujemy obecny jako 202112(tydzień zaciągnięcia danych),
#docelowo może zostać zamieniony funkcją obliczającą jak poniżej
#WPISAĆ FUNKCJĘ!!!!!!!!!

current_week = 202112


In [7]:
#Zmiana terminu rozpoczęcia oraz zakończenia na obecny jeżeli miało mieć ono miejsce w przeszłości
obciazenie_df2.loc[obciazenie_df2['Finnish_week'] < current_week, 'Finnish_week'] = current_week

obciazenie_df2.loc[obciazenie_df2['Pre_fabrication_plan'] < current_week, 'Pre_fabrication_plan'] = current_week

In [8]:
#Zastapienie NaN w godzinach wartościami przewidzianymi z prekalkulacji

#A - dziób(wędka)
#Labor rate	130 rbg/1000kg
#Assembling	0,4
#Welding	0,33
#Grindining	0,07
#Else		0,2

#B - belka(środek)
#Labor rate	100 rbg/1000kg
#Assembling	0,3
#Welding	0,4
#Grindining	0,2
#Else		0,1

#C - końcóka(środek)
#Labor rate	120 rbg/1000kg
#Assembling	0,25
#Welding	0,4
#Grindining	0,2
#Else		0,15

# IF Wszystkie technol = NaN -> Weight for 1 * Quantity * Typ (IF A - 0,13 elif B - 0,1 elif C - 0,12)

In [9]:
#Obliczenie kolumn - pozostałego czasu na produkcję oraz pozostałych godzin
obciazenie_df2['Weeks_left'] = (obciazenie_df2['Finnish_week']) - (obciazenie_df2['Pre_fabrication_plan'])

#Jeżeli >52 to należy odjąc od wyniku 48 (100 - (ilość tyg w danym roku - def 52))
#Może tak się stać dla projektów na przełomie roku 202206-202146 = 60!
obciazenie_df2.loc[obciazenie_df2['Weeks_left'] > 52, 'Weeks_left'] = obciazenie_df2['Weeks_left'] - 48

#Jeżeli zlecenie jest opóźnione i wynik pozostałego czasu będzie wynosił 0
#to zostanie on doliczony do obecnego tygodnia pozdzielony przez 1
obciazenie_df2.loc[obciazenie_df2['Weeks_left'] < 1, 'Weeks_left'] = 1

#funkcja if niestety nie znalazła odpowiedniego zastosowania - loc jest polecany jako dobra praktyka

In [10]:
#???????Czy liczyć nadwyżkę 20% jako zabezpieczenie przy przekroczeniach??????
obciazenie_df2['Work_h_left'] = (obciazenie_df2['Wszystkie.technol']) - (obciazenie_df2['Wszystkie.wykonane'])

#Jeżeli zlecenie zlecenie przekroczyło godziny praca liczona jest w nadgodzinach - przyjmuje się 0 do obciażenia
obciazenie_df2.loc[obciazenie_df2['Work_h_left'] < 0, 'Work_h_left'] = 0

In [11]:
#Podzielenie pozostałej pracy równomiernie na tygodnie
obciazenie_df2['Workload_per_week'] = (obciazenie_df2['Work_h_left']) / (obciazenie_df2['Weeks_left'])
obciazenie_df2['Workload_per_week']

0      0.000000
1      0.000000
2      0.000000
3      7.529412
4     64.700000
        ...    
58          NaN
59          NaN
60          NaN
61          NaN
62          NaN
Name: Workload_per_week, Length: 63, dtype: float64

In [12]:
#Stworzenie finalnej tabeli
obciazenie_df3 = obciazenie_df2[["No", "Pre_fabrication_plan", "Finnish_week", "Workload_per_week"]]

#Stworzenie tabeli z numerami tygodni objętych analizą
x = current_week

Weeks = np.arange(14)
Weeks = current_week
Week_by_week = np.arange(14)
Weeks += Week_by_week

Capacity = obciazenie_df3.reindex(columns = Weeks)

#Połączenie tabel w jedną
#Ustawienie kolejności kolumn
#Zamiana Nan na 0

obciazenie_df4 = pd.concat([Capacity, obciazenie_df3])
obciazenie_df4 = obciazenie_df4.fillna(0)
obciazenie_df4 = ~obciazenie_df4.loc[obciazenie_df4['No'].isin([0])]
obciazenie_df4.set_index('No')
obciazenie_df4




#Podział obciążenia na tygodnie obecny+13 tyg


TypeError: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
# Dodanie obciazenia zarezerwowanego na dany tydzien w zależności od rozpoczecia i zakonczenia danego zlecenia
#If prefabrication plan is >column index & <= finnish week -> column index.No zlecenia = Workload_per_week

#### Może załatwić to iloc 'iem???
####obciazenie_df3.loc[obciazenie_df3.iloc[4] >= ['Pre_fabrication_plan'], obciazenie_df3.iloc[4]] = ['Workload_per_week']
####obciazenie_df3.loc[obciazenie_df3.iloc[4] > ['Finnish_week'], obciazenie_df3.iloc[4]] = 0

#najpierw standardowo
#obciazenie_df3.loc[obciazenie_df3['202112'] >= ['Pre_fabrication_plan'], obciazenie_df3['202112']] = ['Workload_per_week']

In [ ]:
#Zrobienie wykresu